In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install --quiet keybert
print('pip install keybert complete')

env: TOKENIZERS_PARALLELISM=false
pip install keybert complete


In [2]:
import pandas as pd
filename = '/kaggle/input/marcel-proust-in-search-of-lost-time/proust_dataset_ENG.csv'
df = pd.read_csv(filepath_or_buffer=filename, sep='@', index_col=[0])
df['token count'] = df['paragraph'].str.split().apply(len)
df['short paragraph'] = df['paragraph'].apply(func=lambda x: ' '.join(x.split()[:20]))
df = df[df['token count'] > 3] # drop short stuff
df.head()

,paragraph,volume,chapter,token count,short paragraph
0,For a long time I used to go to bed early. Som...,1,1,262,For a long time I used to go to bed early. Som...
1,I would ask myself what o’clock it could be; I...,1,1,116,I would ask myself what o’clock it could be; I...
2,I would lay my cheeks gently against the comfo...,1,1,171,I would lay my cheeks gently against the comfo...
3,"I would fall asleep, and often I would be awak...",1,1,201,"I would fall asleep, and often I would be awak..."
4,"Sometimes, too, just as Eve was created from a...",1,1,199,"Sometimes, too, just as Eve was created from a..."


In [3]:
from plotly.express import histogram
histogram(data_frame=df, x='token count', log_y=True, color='volume')

In [4]:
histogram(data_frame=df, x='volume')

It isn't surprising that a seven-volume work would have volumes that were apparently substantially different lengths, but it is kind of surprising that they would vary this much. By counting paragraphs are we measuring the right thing?

In [5]:
histogram(data_frame=df[['volume', 'token count']].groupby(by='volume').sum().reset_index(), x='volume', y='token count', nbins=7)

If we count by tokens instead of paragraphs we still see divergence between the earlier and later volumes but the difference is not so pronounced.

In [6]:
from arrow import now
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

COLUMN = 'paragraph'
MIN_DF = 2
MODEL = 'all-MiniLM-L12-v2'
STOP_WORDS = 'english'

model_start = now()
model = KeyBERT(model=MODEL,)
# if we set this to 512 we get about 99% of the input intact
model.max_seq_length = 512
raw_documents = df[COLUMN].values
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=STOP_WORDS, min_df=MIN_DF,)
document_embeddings, word_embeddings = model.extract_embeddings(docs=raw_documents, vectorizer=vectorizer, )
print('embedding time: {}'.format(now() - model_start))
print('we have {} documents and {} words.'.format(len(document_embeddings), len(word_embeddings)))
keywords = model.extract_keywords(docs=raw_documents, top_n=1, stop_words=STOP_WORDS, vectorizer=vectorizer,
                                  doc_embeddings=document_embeddings, word_embeddings=word_embeddings, min_df=MIN_DF, )
print('model time: {}'.format(now() - model_start))
df['keyword/weight'] = keywords
df['keyword'] = df['keyword/weight'].apply(func=lambda x: x[0][0] if len(x) else '')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

embedding time: 0:07:25.569708
we have 4330 documents and 48856 words.
model time: 0:07:39.860994


In [7]:
from plotly.express import scatter
from umap import UMAP

umap_start = now()
umap_model = UMAP(n_components=2, random_state=2024, verbose=False, n_jobs=1)
df[['u0', 'u1']] = umap_model.fit_transform(X=document_embeddings)
scatter(data_frame=df, x='u0', y='u1', hover_name='short paragraph', height=900, color='volume', 
       hover_data=['chapter', 'token count', 'keyword']).show()
print('UMAP time: {}'.format(now() - umap_start))

UMAP time: 0:00:45.436058


At a glance it looks like the earlier volumes and the later volumes are talking about different things; it might not be a stretch to say that this graph shows that Volume 1 introduces subject matter that the rest of the series elaborates further.

In [8]:
print(df['keyword'].value_counts().head(n=51).index.tolist())

['saint loup', 'swann', 'mme guermantes', 'albertine', 'duchess', 'duchesse guermantes', 'françoise', 'guermantes', 'mme villeparisis', 'princesse guermantes', 'mme verdurin', 'bloch', 'odette', 'gilberte', 'villeparisis', 'verdurin', 'grandmother', 'duc guermantes', 'princesse parme', 'mme swann', 'norpois', 'aunt', 'legrandin', 'mamma', 'elstir', 'robert', 'mistress', 'flowers', 'forcheville', 'mlle vinteuil', '', 'swanns', 'mme marsantes', 'marquise', 'bergotte', 'saint euverte', 'des laumes', 'sleep', 'doctor', 'vinteuil', 'verdurins', 'pleasures', 'love albertine', 'father', 'brichot', 'edition published', 'duchesse', 'princess', 'charlus', 'jealousy', 'mother']


The keywords are mostly the names of characters; this is not entirely surprising as this is a character-driven story about French society, more or less.

In [9]:
top_keywords = [item for item in df['keyword'].value_counts().head(n=51).index.tolist() if len(item)]
scatter(data_frame=df[df['keyword'].isin(top_keywords)], x='u0', y='u1', hover_name='short paragraph', height=1100,
        color='keyword', hover_data=['chapter', 'token count', 'keyword']).show()

If we use the same scatter plot as before but focus on the top keywords we see that the novel seems to center primarily around Saint-Loup, the Verdurins, and the Guermantes family. Can we get this same information from our data in some other, hopefully simpler, way?

In [10]:
from plotly.express import density_heatmap
density_heatmap(data_frame=df[df['keyword'].isin(top_keywords)][['keyword', 'volume']], y='volume', x='keyword', color_continuous_scale='Blues').show()

The keyword vs. volume density plot is pretty good, actually, as it captures progress through the novel in a way the scatter plots do not.

In [11]:
scatter(data_frame=df[df['keyword'].isin(top_keywords)][['volume', 'keyword']].value_counts().reset_index(), x='keyword', y='volume', color='count',
       color_continuous_scale='Spectral')

The scatter plot does a pretty good job too, but unfortunately it does a poor job of distinguishing among small values.

A reasonable conclusion might be that the current model does a good job of telling us who the novel is about; it doesn't tell us much about what the characters do. In other words, it gives us an entirely noun-driven and mostly proper-noun-driven rather than verb-driven perspective on the text.